In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader, TensorDataset

# from torch.utils.tensorboard import SummaryWriter

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
print(f'Using device: {device}')
# lam = 0.4  


class ResidualBlock(nn.Module):
    def __init__(self, in_features, out_features):
        super(ResidualBlock, self).__init__()
        self.fc1 = nn.Linear(in_features, out_features)
        self.fc2 = nn.Linear(out_features, out_features)
        self.activation = torch.sin
        
        if in_features != out_features:
            self.shortcut = nn.Linear(in_features, out_features)
        else:
            self.shortcut = nn.Identity()

    def forward(self, x):
        identity = self.shortcut(x)
        out = self.activation(self.fc1(x))
        out = self.fc2(out)
        out += identity
        out = self.activation(out)
        return out

class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
        
        # self.hidden_layer1 = ResidualBlock(1, 24)
        # self.hidden_layer2 = ResidualBlock(24, 17)
        # self.hidden_layer3 = ResidualBlock(17, 10)
        # self.hidden_layer4 = ResidualBlock(10, 3)
        # self.hidden_layer5= ResidualBlock(3, 3)

        self.hidden_layer1 = nn.Linear(1, 20)
        self.hidden_layer2 = nn.Linear(20, 20)
        self.hidden_layer3 = nn.Linear(20, 20)
        self.hidden_layer4 = nn.Linear(20, 20)

        # self.hidden_layer1 = nn.Linear(1, 24)
        # self.hidden_layer2 = nn.Linear(24, 17)
        # self.hidden_layer3 = nn.Linear(17, 10)
        # self.hidden_layer4 = nn.Linear(10, 3)
        # self.hidden_layer5 = nn.Linear(3, 3)
        self.output_layer = nn.Linear(20, 1)
        self.learned = False 
        self.log_lam = torch.tensor(0.0)

    def forward(self, y):
        y = torch.tanh(self.hidden_layer1(y))
        y = torch.tanh(self.hidden_layer2(y))
        y = torch.tanh(self.hidden_layer3(y))
        y = torch.tanh(self.hidden_layer4(y))
        # y = torch.tanh(self.hidden_layer5(y))
        y = self.output_layer(y)
        return y
    
    def U(self, y):
        return (self(y) - self(-y)) / 2

    def get_lam(self, y):
        if self.learned:
            U = self.U(y)
            U_y = torch.autograd.grad(U, y, grad_outputs=torch.ones_like(U), create_graph=True)[0]
            U_yy = torch.autograd.grad(U_y, y, grad_outputs=torch.ones_like(U_y), create_graph=True)[0]
            return torch.mean(torch.divide(y*U_yy , -(1 + U_y) * U_y - (U + y)*U_yy))
        return .4
    
def f(y,U,U_y,lam):
    return -lam * U + ((1 + lam) * y + U) * U_y

def compute_derivative(f, y, model, lam, orders,finite=False):
    y.requires_grad = True
    U = model.U(y)
    U_y = torch.autograd.grad(U, y, grad_outputs=torch.ones_like(U), create_graph=True)[0]
    lam = model.get_lam(y)
    f_val = f(y, U, U_y, lam)
    h = y[1] - y[0]
    res = []
    if not finite:
        for _ in range(int(orders.max())):
            f_val = torch.autograd.grad(f_val, y, grad_outputs=torch.ones_like(f_val), create_graph=True)[0]
            if _ + 1 in orders:
                res.append(f_val)
    else:
        for _ in range(int(orders.max())):
            f_val = (y[1:] - y[:-1]) / h
            if _ + 1 in orders:
                res.append(f_val)
    return res

def Loss(model, y, collocation_points):
    y.requires_grad = True
    U = model.U(y)
    U_y = torch.autograd.grad(U, y, grad_outputs=torch.ones_like(U), create_graph=True)[0]
    lam = model.get_lam(y)

    # Equation loss
    f_val = f(y, U, U_y,lam)

    # Smooth loss 3rd and fifth derivative
    derivatives = compute_derivative(f,collocation_points,model,lam,np.array([3.0]),True)
    f_yyy = derivatives[0]
    # f_yyyyy = derivatives[1]
 

    # Condition loss U(-2) = 1
    g = model.U(torch.tensor([-2.0], dtype=y.dtype, device=y.device)) - 1
    
    equation_loss = torch.mean(f_val**2)
    condition_loss = torch.mean(g**2)

    total_loss = equation_loss + condition_loss + 1e-3*torch.mean(f_yyy**2) #+ 1e-5*torch.mean(f_yyyyy**2)
    return total_loss

Using device: cpu


### Fixed lambda 

In [2]:
# Initialize model 

model = PINN().to(device)
# model = torch.compile(model)

optimizer = optim.LBFGS(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.99)

# Training parameters
num_epochs = 100
batch_size = 128 
y_data = torch.linspace(-2,2,10000).view(-1, 1).to(device)

Ns = 1000
collocation_points = torch.FloatTensor(Ns).uniform_(-1, 1).view(-1, 1).to(device)
collocation_points = (collocation_points - collocation_points.mean()) / collocation_points.std()

# Create DataLoader for y_data and collocation_points
y_dataset = TensorDataset(y_data)
collocation_dataset = TensorDataset(collocation_points)
y_loader = DataLoader(y_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
collocation_loader = DataLoader(collocation_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

def closure(y_batch, collocation_batch):
    optimizer.zero_grad()  
    loss = Loss(model, y_batch, collocation_batch)  
    loss.backward()  
    return loss

for epoch in range(num_epochs):
    for y_batch, collocation_batch in zip(y_loader, collocation_loader):
        y_batch = y_batch[0].to(device)
        collocation_batch = collocation_batch[0].to(device)
        optimizer.step(lambda: closure(y_batch, collocation_batch)) #
    
    if epoch % 1 == 0:
        y_batch = next(iter(y_loader))[0].to(device)
        collocation_batch = next(iter(collocation_loader))[0].to(device)
        loss = Loss(model, y_batch, collocation_batch)
        print(f'epoch {epoch}, loss {loss.item()}')
        if loss.item() <= 1e-8:
            break

epoch 0, loss 22.626935958862305
epoch 1, loss 0.021237915381789207
epoch 2, loss 0.010468811728060246
epoch 3, loss 0.009003899991512299
epoch 4, loss 0.008861695416271687
epoch 5, loss 0.012052849866449833
epoch 6, loss 0.00833094958215952
epoch 7, loss 0.02335018664598465
epoch 8, loss 0.03339057415723801
epoch 9, loss 0.008581708185374737
epoch 10, loss 0.01116197369992733
epoch 11, loss 0.008121078833937645
epoch 12, loss 0.013945475220680237
epoch 13, loss 0.2813381552696228
epoch 14, loss 0.049136821180582047
epoch 15, loss 0.01099590864032507
epoch 16, loss 0.008149244822561741
epoch 17, loss 0.016965333372354507
epoch 18, loss 0.009786972776055336
epoch 19, loss 0.015034589916467667
epoch 20, loss 0.011710010468959808
epoch 21, loss 0.008820496499538422
epoch 22, loss 0.008811186999082565
epoch 23, loss 0.1055724024772644


KeyboardInterrupt: 

### Lambda learned in the process 

In [ ]:
model.learned = True
for epoch in range(num_epochs):
    for y_batch, collocation_batch in zip(y_loader, collocation_loader):
        y_batch = y_batch[0].to(device)
        collocation_batch = collocation_batch[0].to(device)
        optimizer.step(lambda: closure(y_batch, collocation_batch))
    
    if epoch % 1 == 0:
        y_batch = next(iter(y_loader))[0].to(device)
        collocation_batch = next(iter(collocation_loader))[0].to(device)
        loss = Loss(model, y_batch, collocation_batch)
        print(f'epoch {epoch}, loss {loss.item()}')
        if loss.item() <= 1e-8:
            break

In [ ]:
y_test = 2*torch.sin(torch.linspace(-np.pi/2, np.pi/2, 100)).view(-1, 1).to(device)
y_test.requires_grad = True
# Get model predictions and detach to move to CPU
U_pred = model.U(y_test)
U_pred_y = torch.autograd.grad(U_pred, y_test, grad_outputs=torch.ones_like(U_pred), create_graph=True)[0]
U_pred_yy = torch.autograd.grad(U_pred_y, y_test, grad_outputs=torch.ones_like(U_pred_y), create_graph=True)[0]
U_pred_yyy = torch.autograd.grad(U_pred_yy, y_test, grad_outputs=torch.ones_like(U_pred_yy), create_graph=True)[0]
U_pred_yyyy = torch.autograd.grad(U_pred_yyy, y_test, grad_outputs=torch.ones_like(U_pred_yyy), create_graph=True)[0]
U_pred_yyyyy = torch.autograd.grad(U_pred_yyyy, y_test, grad_outputs=torch.ones_like(U_pred_yyyy), create_graph=True)[0]

lam = model.get_lam(y_test).detach().cpu().numpy()
print(lam)
residual = f(y_test,U_pred,U_pred_y,lam)
print(torch.sqrt(torch.mean(residual**2)))
U_pred = U_pred.detach().cpu().numpy()
# Generate exact solution using implicit formula
U_positive = np.linspace(0, 1, 100)
y_true = np.array([U_positive + U_positive**(1 + 1/lam), -U_positive - U_positive**(1 + 1/lam)]).flatten()
order = y_true.argsort()
U_sorted = np.array([-U_positive, U_positive]).flatten()[order]

y_sorted = y_true[order]

# Convert test data to numpy
y_test_np = y_test.detach().cpu().numpy()

# Plotting
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot the PINN Prediction vs Exact Solution
ax1.plot(y_test_np, U_pred, '.-', label='PINN Prediction', color='#1f77b4', markersize=5)
ax1.plot(y_sorted, U_sorted, label='Exact Solution', color='#ff7f0e', linestyle='--', linewidth=2)
ax1.set_title('Comparison of PINN Prediction and Exact Solution')
ax1.set_xlabel('y')
ax1.set_ylabel('U')
ax1.grid(True, which='both', linestyle='--', linewidth=0.5)
ax1.legend()

# Plot the third derivative
ax2.plot(y_test_np, U_pred_yyy.detach().cpu().numpy(), '.-', label='Third Derivative of U', color='#2ca02c', markersize=5)
ax2.set_title('Third Derivative of U')
ax2.set_xlabel('y')
ax2.set_ylabel('d^3U/dy^3')
ax2.grid(True, which='both', linestyle='--', linewidth=0.51)
ax2.legend()

plt.tight_layout()
plt.show()